In [1]:
# %pip install numpy
# %pip install -q mediapipe
# %pip install opencv-camera

In [ ]:
%%html --isolated
<h2>TODO:</h2>
<ul>
    <li>Leverage mp solutions to map predefined gestures to actions</li>
    <li>Create importer class to allow user to determine gestures</li>
    <li>Clean up code, include commentation</li>
</ul>

In [3]:
import numpy as np
import cv2
import mediapipe as mp 
from mediapipe.framework.formats import landmark_pb2
import time

# from mediapipe.tasks import python
# from mediapipe.tasks.python import vision
# from mediapipe import solutions

In [4]:
finger_tips = [4,8,12,16,20] 

In [5]:
def draw_landmarks_on_image(rgb_image, detection_result: mp.tasks.vision.HandLandmarkerResult):
    try:
        if detection_result.hand_landmarks == []:
            print("No landmarks to draw")
            return rgb_image
        else:
            hand_landmarks_list = detection_result.hand_landmarks
            annotated_image = np.copy(rgb_image)

            # Loop through the detected hands to visualize.
            for idx in range(len(hand_landmarks_list)):
                hand_landmarks = hand_landmarks_list[idx]
            
                # Draw the hand landmarks.
                hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
                hand_landmarks_proto.landmark.extend(
                    [landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks]
                )
                mp.solutions.drawing_utils.draw_landmarks(
                    annotated_image,
                    hand_landmarks_proto,
                    mp.solutions.hands.HAND_CONNECTIONS,
                    mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                    mp.solutions.drawing_styles.get_default_hand_connections_style()
                )
            return annotated_image
    except Exception as e:
        print(e)
        return rgb_image

In [6]:
# wrapper class to make accessing the landmarks-model and its results easier
class landmarker_and_result():
    def createLandmarker(self):
        def update_result(result: mp.tasks.vision.HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
            self.result = result
            
        options = mp.tasks.vision.HandLandmarkerOptions( 
            base_options = mp.tasks.BaseOptions(model_asset_path="./hand_landmarker.task"), # path to model
            running_mode = mp.tasks.vision.RunningMode.LIVE_STREAM, # running on a live stream
            num_hands = 1, # track one hand
            min_hand_detection_confidence = 0.5, # lower the value to get predictions more often
            min_hand_presence_confidence = 0.5, # lower the value to get predictions more often
            min_tracking_confidence = 0.3, # lower the value to get predictions more often
            result_callback=update_result)
        # initialize landmarker
        self.landmarker = self.landmarker.create_from_options(options)
    
    def detect_async(self, frame):
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data = frame)
        self.landmarker.detect_async(image = mp_image, timestamp_ms = int(time.time() * 1000))
    
    def close(self):
        self.landmarker.close()

    def __init__(self):
        self.result = mp.tasks.vision.HandLandmarkerResult
        self.landmarker = mp.tasks.vision.HandLandmarker
        self.createLandmarker()

In [7]:
def count_up_down(result: mp.tasks.vision.HandLandmarkerResult):
    if(not result):
        print("No results to count")
        return [-1,-1]
    try:
        hand_landmarks_list = result.hand_landmarks
    except Exception as e:
        print(e)
        print("No landmarks to count fingers")
        return [-1,-1]
    
    if(hand_landmarks_list == []):
        print("No landmarks preset")
        return [-1,-1]
    
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend(
        [landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks_list[0]]
    )
    try:
        normal_landmarks_list = hand_landmarks_proto.landmark
        up = 0
        down = 0
        # print('len of hand_landmarks: ', len(hand_landmarks_list), '\nlen of normalized list:', len(normal_landmarks_list))
        #skipping thumb since this method doesn't work well with it
        for pos in range(8,21,4):
            print(pos)
            if(normal_landmarks_list[pos].y < normal_landmarks_list[pos-3].y):
                up = up + 1
            else:
                down = down + 1
        return [up,down]
    except Exception as e:
        print(e)
    return [-1,-1]

In [8]:
try:
    cv2.namedWindow("preview")
    vc = cv2.VideoCapture(0)

    if vc.isOpened():
        rval, frame = vc.read()
    else: 
        print("Could not open camera")
        rval = False

    hand_landmarker = landmarker_and_result()
    hand_landmarker.detect_async(frame)

    time.sleep(.5)

    while rval:
        ret, frame = vc.read()
        if(not ret): continue
        
        frame = cv2.flip(frame, 1)
        hand_landmarker.detect_async(frame)
        frame = draw_landmarks_on_image(frame, hand_landmarker.result)
        
        print(count_up_down(hand_landmarker.result))
        
        cv2.imshow("preview", frame)

        key = cv2.waitKey(1)
        if key == 27:
            break
except Exception as e:
    print(e)
    print("closing out...")
    
hand_landmarker.close()
vc.release()
cv2.destroyWindow("preview")



No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
No landmarks to draw
No landmarks preset
[-1, -1]
